In [119]:
import pandas as pd
import numpy as np
np.random.seed(1)
from functools import partial
import itertools

import plotly.graph_objects as go
import plotly.express as px

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS
import sklearn.metrics as metrics

ponte = pd.read_pickle(r'Data\Data_Ponte_dos_Remedios.pkl')
del ponte['o3']
guarulhos = pd.read_pickle(r'Data\Data_Guarulhos.pkl')
guarulhos = guarulhos[['date','o3']]

data = ponte.merge(guarulhos, on='date', how='outer')
data.reset_index(drop=True)

import shutil
import pickle
from IPython.display import clear_output
import os
os.environ['NIXTLA_ID_AS_COL'] = '1'

from pytorch_lightning import Trainer
trainer = Trainer(
    max_steps=4,
    logger=False,
    enable_progress_bar=False,
    enable_model_summary=False  # Disable model summary
)

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="optuna")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


## GUMODEX

In [6]:
# USED WITH PLOTLY TO ELABORATE THE DESIGN ===========================================================
def main_layout(fig:go.Figure, width=700, height=600, x=None, y=None, title=None,
               x_range=None, y_range=None, paper_color='white', 
               customdata=None, hover_customdata='Info', 
               hover_x='x',hover_y='y', **kwargs) -> go.Figure:
    fig.layout = go.Layout(
        width=width,
        height=height,
        plot_bgcolor=paper_color,
        paper_bgcolor=paper_color,
        xaxis={'gridcolor':'#cccccc', 'linecolor':'black','title':x, 'range':x_range},
        yaxis={'gridcolor':'#cccccc', 'linecolor':'black','title':y, 'range':y_range},
        title={'text':title},
        **kwargs
    )
    if customdata == 'no':
        ...
    elif customdata is None:
        fig.update_traces(patch={
            'customdata':customdata, 'hovertemplate': hover_x + ': %{x}<br>' + hover_y + ': %{y}'
        })
    else:
        fig.update_traces(patch={
            'customdata':customdata,
            'hovertemplate': hover_x + ': %{x}<br>' + hover_y + ': %{y}<br>' + hover_customdata + ': %{customdata}<br>'
        })
    return fig
# ====================================================================================================
def main_subplot_layout(fig:go.Figure, width=1400, height=500, title=None, paper_color='white',
                        x=None, y=None, rows=1, cols=2, x_range=None, y_range=None,
                        customdata=None, hover_customdata='Info', 
                        hover_x='x',hover_y='y', **kwargs) -> go.Figure:
    fig.update_layout({
        'width':width,
        'height':height,
        'plot_bgcolor':paper_color,
        'paper_bgcolor':paper_color,
        'title':title,
        **kwargs
    })
    for xaxis in fig.select_xaxes():
        xaxis.update(
            showgrid=True,
            gridcolor='#CCCCCC',
            linecolor='black',
            title=x,
            range=x_range
        )
    for yaxis in fig.select_yaxes():
        yaxis.update(
            showgrid=True,
            gridcolor='#CCCCCC',
            linecolor='black',
            title=y,
            range=y_range
        )
    if customdata == 'no':
        ...
    elif customdata is None:
        fig.update_traces(patch={
            'customdata':customdata, 'hovertemplate': hover_x + ': %{x}<br>' + hover_y + ': %{y}'
        })
    else:
        fig.update_traces(patch={
            'customdata':customdata,
            'hovertemplate': hover_x + ': %{x}<br>' + hover_y + ': %{y}<br>' + hover_customdata + ': %{customdata}<br>'
        })
    return fig
# ====================================================================================================

# **Time Object**

In [8]:
class TimeObject:
    def __init__(self, df:pd.DataFrame, column:str, 
                 NAN_treatment_args:dict={'method':'from_derivatives'},
                 agg_freq:str=None) -> None:

        self.df = df[['date',column]]
        self.column = column
        self.time_serie = self.to_serie_()

        self.NAN_treatment_(**NAN_treatment_args)
        self.NIXTLA_treatment_()
        if agg_freq != None: 
            self.nixtla_df = self.nixtla_df.groupby(pd.Grouper(key='ds', freq=agg_freq)).agg({'y': 'mean'}).reset_index()
            self.nixtla_df.loc[:, ['unique_id']] = 1.0
        self.NIXTLA_train_test(split=7)

    def to_serie_(self) -> pd.Series:
        time_serie = self.df[self.column].fillna(np.nan)
        time_serie.index = pd.to_datetime(self.df['date'])

        full_index = pd.date_range(start=time_serie.index.min(), end=time_serie.index.max(), freq='D')
        time_serie = time_serie.reindex(full_index)
        return time_serie
    
    def NAN_treatment_(self, **kwargs) -> None:
        self.time_serie = self.time_serie.interpolate(**kwargs)
    
    def NIXTLA_treatment_(self) -> None:
        self.nixtla_df = pd.DataFrame()
        self.nixtla_df.loc[:, ['ds']] = pd.to_datetime(self.time_serie.index)
        self.nixtla_df.loc[:, ['y']] = self.time_serie.values
        self.nixtla_df.loc[:, ['unique_id']] = 1.0

    def plot(self) -> go.Figure:
        fig = go.Figure()
        fig.add_trace(trace=go.Scatter(
            x=self.time_serie.index, y=self.time_serie,
            marker=dict(color='#222222')
        ))
        return fig

    def NIXTLA_train_test(self, split:int=12):
        self.split = split
        self.Y_train = self.nixtla_df[self.nixtla_df.ds<self.nixtla_df['ds'].values[-split]]
        self.Y_test = self.nixtla_df[self.nixtla_df.ds>=self.nixtla_df['ds'].values[-split]].reset_index(drop=True)

    def metrics_(self, forecast_df:pd.DataFrame, method:str='NHITS'):

        def smape(y_true, y_pred):
            summation = 0
            for i in range(len(y_true)):
                summation += np.abs(y_true[i]-y_pred[i])/(np.abs(y_true[i]) + np.abs(y_pred[i]))
            return 200/(len(y_true)+1) * summation
        
        self.metrics = {}
        self.metrics['mae'] = np.round(metrics.mean_absolute_error(y_true=self.Y_test['y'], y_pred=forecast_df[method]),5)
        self.metrics['mape'] = np.round(100*metrics.mean_absolute_percentage_error(y_true=self.Y_test['y'], y_pred=forecast_df[method]),5)
        self.metrics['mse'] = np.round(metrics.mean_squared_error(y_true=self.Y_test['y'], y_pred=forecast_df[method]),5)
        self.metrics['max'] = np.round(metrics.max_error(y_true=self.Y_test['y'], y_pred=forecast_df[method]),5)
        self.metrics['smape'] = np.round(smape(y_true=self.Y_test['y'], y_pred=forecast_df[method]),5)
        return

    def plot_time_series(self):
        fig = go.Figure()
        fig.add_trace(trace=go.Scatter(
            x=self.Y_train['ds'], y=self.Y_train['y'],
            mode='lines', marker=go.scatter.Marker(
                color='black'
            ), name='Time Series'
        ))
        main_layout(fig=fig, width=1100, height=450, title='Time Series', x='time', y='AQI')
        fig.show()

    def plot_forecast(self, forecast_df:pd.DataFrame, confidence:int=90, method='NHITS', show:bool=True, show_metrics:bool=True):
        fig = go.Figure()

        fig.add_trace(trace=go.Scatter(
            x=self.Y_train['ds'], y=self.Y_train['y'],
            mode='lines', marker=go.scatter.Marker(
                color='black'
            ), name='train'
        ))
        
        fig.add_trace(trace=go.Scatter(
            x=self.Y_test['ds'], y=self.Y_test['y'],
            mode='lines', marker=go.scatter.Marker(
                color='skyblue'
            ), name='test'
        ))

        fig.add_trace(trace=go.Scatter(
            x=forecast_df['ds'], y=forecast_df[f'{method}'],
            mode='lines', marker=go.scatter.Marker(
                color='orange'
            ), name=method
        ))

        try:
            fig.add_trace(go.Scatter(
                x=forecast_df['ds'], y=forecast_df[f'{method}-lo-{confidence}'],
                mode='lines', line=dict(width=0), fill='tonexty',
                fillcolor='rgba(255, 165, 0, 0)',
                showlegend=False
            ))

            fig.add_trace(go.Scatter(
                x=forecast_df['ds'], y=forecast_df[f'{method}-hi-{confidence}'],
                mode='lines', line=dict(width=0), fill='tonexty',
                fillcolor='rgba(255, 165, 0, 0.2)',
                name=f'confidence: {confidence}%'
            ))
        except: ...

        main_layout(fig=fig, width=1100, height=450, title='Forecast', x='time', y='AQI')

        if show:
            fig.show()
        if show_metrics:
            self.metrics_(forecast_df, method=method)
            for key, metric in self.metrics.items():
                print(f'{key}: {metric}')
        
        return fig

# **NHiTS**

### One

In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.losses.pytorch import DistributionLoss

obj = TimeObject(df=ponte[:], column='pm25', agg_freq='W')
obj.NIXTLA_train_test(split=52)

model = NHITS(h=52,
              input_size=104,
              loss=DistributionLoss(distribution='StudentT', level=[95], return_params=True),
              stack_types=4*['identity'],
              n_freq_downsample=(52, 12, 4, 1),
              n_blocks=4*[3],
              n_pool_kernel_size=[3, 2, 1, 1],
              pooling_mode="MaxPool1d",
              activation="ReLU",
              scaler_type='identity',
              interpolation_mode='linear',
              max_steps=150,
              val_check_steps=12)

fcst = NeuralForecast(
    models=[model],
    freq='W',
    local_scaler_type='boxcox'
)
fcst.fit(df=obj.Y_train, verbose=False)
prediction = fcst.predict(df=obj.Y_train)

fig = obj.plot_forecast(forecast_df=prediction, confidence=95)

### Optuna

In [23]:
import optuna
import pandas as pd

# Initialize the results list
results = []

# Define the objective function
def objective(trial):
    # Hyperparameter search space
    input_size = trial.suggest_int('input_size', 4, 104, step=1)
    n_stacks = trial.suggest_int('n_stacks', 3, 7, step=1)
    n_blocks = trial.suggest_int('n_blocks', 1, 5, step=1)
    max_steps = trial.suggest_int('max_steps', 1, 500, step=1)
    local_scalar_type = trial.suggest_categorical('local_scalar_type', [None, 'standard', 'boxcox', 'minmax'])
    n_pool_kernel_size = trial.suggest_categorical('n_pool_kernel_size', [list(combination) for combination in list(itertools.product([1, 2, 3], repeat=3))])
    n_freq_downsample = trial.suggest_categorical('n_freq_downsample', [list(combination) for combination in list(itertools.product([1, 4, 12, 52], repeat=3))])

    #choose split for cross validation
    split = [1, 30, 90, 180, 365, 730]
    split = np.random.choice(split)  
    mape = []
    smape = []
    for pollutant in data[['pm10','pm25','co','no2','o3']]:
        # Instantiate TimeObject and prepare training data
        obj = TimeObject(df=data[:-split], column=pollutant, agg_freq='W')
        obj.NIXTLA_train_test(split=52)

        # Define the model
        model = NHITS(
            h=52,
            input_size=input_size,
            stack_types=n_stacks*['identity'],
            n_freq_downsample=n_freq_downsample+(n_stacks-len(n_freq_downsample))*[1],
            n_blocks=n_stacks*[n_blocks],
            n_pool_kernel_size=(n_stacks-len(n_pool_kernel_size))*[1]+n_pool_kernel_size,
            pooling_mode="MaxPool1d",
            activation="ReLU",
            interpolation_mode='linear',
            max_steps=max_steps,
            val_check_steps=12
        )

        # Initialize NeuralForecast and fit the model
        fcst = NeuralForecast(
            models=[model],
            freq='W',
            local_scaler_type=local_scalar_type
        )
        fcst.fit(df=obj.Y_train, verbose=False)
        prediction = fcst.predict(df=obj.Y_train, verbose=False)

        # Evaluate metrics
        obj.metrics_(forecast_df=prediction, method='NHITS')
        mape.append(obj.metrics['mape'])
        smape.append(obj.metrics['smape'])
        
        clear_output(wait=True)

    directory_path = "lightning_logs"
    if os.path.exists(directory_path):
        shutil.rmtree(directory_path)

    mape = np.mean(mape)
    smape = np.mean(smape)

    # Collect the results
    results.append({
        'freq': 'W',
        'split': split,
        'h': 52,
        'input_size': input_size,
        'n_stacks': n_stacks,
        'n_blocks': n_blocks,
        'max_steps': max_steps,
        'local_scalar_type': local_scalar_type,
        'n_pool_kernel_size': n_pool_kernel_size,
        'n_freq_downsample': n_freq_downsample,
        'mape': mape,
        'smape': smape,
    })

    # The objective for Optuna is to minimize the MAE (or maximize a metric)
    return smape  # You can change this to any metric you want to optimize

# Define the optimization study_nhits
study_nhits = optuna.create_study(direction='minimize')  # Minimize the MAE

# Run the optimization with the number of trials you want
study_nhits.optimize(objective, n_trials=150)

clear_output(wait=True)
NHITS_W = pd.DataFrame(results)
NHITS_W.to_pickle(r'NHITS_W.pkl')

[I 2024-12-07 02:09:07,058] Trial 149 finished with value: 28.384742 and parameters: {'input_size': 44, 'n_stacks': 3, 'n_blocks': 2, 'max_steps': 119, 'local_scalar_type': 'boxcox', 'n_pool_kernel_size': [3, 1, 1], 'n_freq_downsample': [1, 12, 52]}. Best is trial 82 with value: 21.310983999999998.


#### Plot

In [24]:
# NBEATS_W.sort_values(by=['mape','smape'])
print(f"Best trial: {study_nhits.best_trial.params}")
fig = optuna.visualization.plot_contour(study_nhits)
fig.update_layout({'height':1000, 'width':1000, 'plot_bgcolor':'white'})
fig.show()

Best trial: {'input_size': 47, 'n_stacks': 3, 'n_blocks': 2, 'max_steps': 160, 'local_scalar_type': 'boxcox', 'n_pool_kernel_size': [1, 3, 2], 'n_freq_downsample': [1, 52, 4]}


#### Try Out

In [25]:
for pollutant in data[['pm10','pm25','co','no2','o3']]:
    obj = TimeObject(df=data, column=pollutant, agg_freq='W')
    obj.NIXTLA_train_test(split=52)

    model = NBEATS(
        h=52,
        input_size=study_nhits.best_trial.params.get('input_size'),
        loss=DistributionLoss(distribution='StudentT', level=[95], return_params=True),
        stack_types=study_nhits.best_trial.params.get('n_stacks')*['identity'],
        n_blocks =[study_nhits.best_trial.params.get('n_blocks') for _ in range(study_nhits.best_trial.params.get('n_stacks'))],
        max_steps=study_nhits.best_trial.params.get('max_steps'),
        val_check_steps=3,
        learning_rate=1e-3
    )
    fcst = NeuralForecast(
        models=[model],
        freq='W',
        local_scaler_type=study_nhits.best_trial.params.get('local_scaler_type')
    )
    fcst.fit(df=obj.Y_train, verbose=False)
    prediction = fcst.predict(df=obj.Y_train, verbose=False)

    fig = obj.plot_forecast(forecast_df=prediction, confidence=None, method='NBEATS', show=False)
    fig.update_layout({'width':1000, 'height':300})
    fig.show()

Seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type             | Params | Mode 
----------------------------------------------------------
0 | loss         | DistributionLoss | 3      | train
1 | padder_train | ConstantPad1d    | 0      | train
2 | scaler       | TemporalNorm     | 0      | train
3 | blocks       | ModuleList       | 5.5 M  | train
----------------------------------------------------------
5.5 M     Trainable params
3         Non-trainable params
5.5 M     Total params
22.000    Total estimated model params size (MB)
58        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=160` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

mae: 5.77496
mape: 29.20365
mse: 50.76402
max: 18.70267
smape: 23.53207


# **N-BEATS**

### One

In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS
from neuralforecast.losses.pytorch import DistributionLoss

obj = TimeObject(df=data, column='o3', agg_freq='W')
obj.NIXTLA_train_test(split=52)

model = NBEATS(h=52,
               input_size=36,
               loss=DistributionLoss(distribution='StudentT', level=[95], return_params=True),
               stack_types=['identity'],
               n_blocks = [3],
               max_steps=100,
               val_check_steps=3,
               learning_rate=1e-3)

fcst = NeuralForecast(
    models=[model],
    freq='W',
    local_scaler_type='standard'
)
fcst.fit(df=obj.Y_train, verbose=False)
prediction = fcst.predict(df=obj.Y_train, verbose=False)

obj.plot_forecast(forecast_df=prediction, confidence=95, method='NBEATS')

### Optuna

In [ ]:
import optuna
import pandas as pd

# Initialize the results list
results = []

# Define the objective function
def objective(trial):
    # Hyperparameter search space
    input_size = trial.suggest_int('input_size', 4, 104, step=1)
    n_stacks = trial.suggest_int('n_stacks', 2, 7, step=1)
    n_blocks = trial.suggest_int('n_blocks', 1, 5, step=1)
    max_steps = trial.suggest_int('max_steps', 1, 500, step=1)
    local_scalar_type = trial.suggest_categorical('local_scalar_type', [None, 'standard', 'boxcox', 'minmax'])
    interpretability = trial.suggest_categorical('interpretability', [['seasonality','trend'],['seasonality','identity'],['trend','identity'],['identity','identity']])

    #choose split for cross validation
    split = [1, 30, 90, 180, 365, 730]
    split = np.random.choice(split)  
    mape = []
    smape = []
    for pollutant in data[['pm10','pm25','co','no2','o3']]:
        # Instantiate TimeObject and prepare training data
        obj = TimeObject(df=data[:-split], column=pollutant, agg_freq='W')
        obj.NIXTLA_train_test(split=52)

        # Define the model
        model = NBEATS(
            h=52,
            input_size=input_size,
            stack_types=interpretability+(n_stacks-len(interpretability))*['identity'],
            n_blocks=n_stacks * [n_blocks],
            max_steps=max_steps,
            val_check_steps=12,
            learning_rate=1e-3
        )

        # Initialize NeuralForecast and fit the model
        fcst = NeuralForecast(
            models=[model],
            freq='W',
            local_scaler_type=local_scalar_type
        )
        fcst.fit(df=obj.Y_train, verbose=False)
        prediction = fcst.predict(df=obj.Y_train, verbose=False)

        # Evaluate metrics
        obj.metrics_(forecast_df=prediction, method='NBEATS')
        mape.append(obj.metrics['mape'])
        smape.append(obj.metrics['smape'])
        
        clear_output(wait=True)

    directory_path = "lightning_logs"
    if os.path.exists(directory_path):
        shutil.rmtree(directory_path)

    mape = np.mean(mape)
    smape = np.mean(smape)

    # Collect the results
    results.append({
        'freq': 'W',
        'split': split,
        'h': 52,
        'input_size': input_size,
        'n_stacks': n_stacks,
        'n_blocks': n_blocks,
        'max_steps': max_steps,
        'local_scalar_type': local_scalar_type,
        'interpretability': interpretability,
        'mape': mape,
        'smape': smape,
    })

    # The objective for Optuna is to minimize the MAE (or maximize a metric)
    return smape  # You can change this to any metric you want to optimize

# Define the optimization study_nbeats
study_nbeats = optuna.create_study(direction='minimize')  # Minimize the MAE

# Run the optimization with the number of trials you want
study_nbeats.optimize(objective, n_trials=150)

clear_output(wait=True)
NBEATS_W = pd.DataFrame(results)
NBEATS_W.to_pickle(r'NBEATS_W.pkl')

[I 2024-12-07 08:14:12,555] Trial 149 finished with value: 29.022712000000002 and parameters: {'input_size': 25, 'n_stacks': 5, 'n_blocks': 5, 'max_steps': 63, 'local_scalar_type': 'boxcox', 'interpretability': ['seasonality', 'trend']}. Best is trial 117 with value: 23.464420000000004.


In [ ]:
import optuna
import pandas as pd
np.random.seed(1)

# Define the objective function
def objective(trial, pollutant):
    # Hyperparameter search space
    input_size = trial.suggest_int('input_size', 4, 104, step=1)
    n_stacks = trial.suggest_int('n_stacks', 2, 7, step=1)
    n_blocks = trial.suggest_int('n_blocks', 1, 5, step=1)
    max_steps = trial.suggest_int('max_steps', 1, 500, step=1)
    local_scalar_type = trial.suggest_categorical('local_scalar_type', [None, 'standard', 'boxcox', 'minmax'])
    interpretability = trial.suggest_categorical('interpretability', [['seasonality','trend'],['seasonality','identity'],['trend','identity'],['identity','identity']])

    #choose split for cross validation
    split = np.random.choice([1, 30, 90, 180, 365, 730])  
    mape = []
    smape = []
    # Instantiate TimeObject and prepare training data
    obj = TimeObject(df=data[:-split], column=pollutant, agg_freq='W')
    obj.NIXTLA_train_test(split=52)

    # Define the model
    model = NBEATS(
        h=52,
        input_size=input_size,
        stack_types=interpretability+(n_stacks-len(interpretability))*['identity'],
        n_blocks=n_stacks * [n_blocks],
        max_steps=max_steps,
        val_check_steps=12,
        learning_rate=1e-3
    )

    # Initialize NeuralForecast and fit the model
    fcst = NeuralForecast(
        models=[model],
        freq='W',
        local_scaler_type=local_scalar_type
    )
    fcst.fit(df=obj.Y_train, verbose=False)
    prediction = fcst.predict(df=obj.Y_train, verbose=False)

    # Evaluate metrics
    obj.metrics_(forecast_df=prediction, method='NBEATS')
    mape.append(obj.metrics['mape'])
    smape.append(obj.metrics['smape'])
    
    clear_output(wait=True)

    directory_path = "lightning_logs"
    if os.path.exists(directory_path):
        shutil.rmtree(directory_path)

    mape = np.mean(mape)
    smape = np.mean(smape)

    # Collect the results
    results.append({
        'pollutant': pollutant,
        'freq': 'W',
        'split': split,
        'h': 52,
        'input_size': input_size,
        'n_stacks': n_stacks,
        'n_blocks': n_blocks,
        'max_steps': max_steps,
        'local_scalar_type': local_scalar_type,
        'interpretability': interpretability,
        'mape': mape,
        'smape': smape,
    })

    # The objective for Optuna is to minimize the MAE (or maximize a metric)
    return smape  # You can change this to any metric you want to optimize

for pollutant in data[['pm10','pm25','co','no2','o3']]:
    # Initialize the results list
    results = []
    # Define the optimization study_nbeats
    study_nbeats = optuna.create_study(direction='minimize')  # Minimize the MAE

    # Run the optimization with the number of trials you want
    study_nbeats.optimize(partial(objective, pollutant=pollutant), n_trials=150)

    clear_output(wait=True)
    NBEATS_W = pd.DataFrame(results)
    NBEATS_W.to_pickle(fr'Results\NBEATS_W_Df_{pollutant}.pkl')
    with open(fr"Results\NBEATS_W_Study_{pollutant}.pkl", "wb") as f:
        pickle.dump(study_nbeats, f)

[I 2024-12-07 15:17:18,138] Trial 1 finished with value: 66.57516 and parameters: {'input_size': 71, 'n_stacks': 4, 'n_blocks': 5, 'max_steps': 1, 'local_scalar_type': 'standard', 'interpretability': ['seasonality', 'identity']}. Best is trial 0 with value: 38.79593.


In [141]:
with open(r'Results\NBEATS_W_Df_no2.pkl','rb') as f:
    loaded = pickle.load(f)

#### Plots

In [29]:
# NBEATS_W.sort_values(by=['mape','smape'])
print(f"Best trial: {study_nbeats.best_trial.params}")
fig = optuna.visualization.plot_contour(study_nbeats, params=['input_size','n_stacks','n_blocks','max_steps','local_scalar_type'])
fig.update_layout({'height':1000, 'width':1000, 'plot_bgcolor':'white'})
fig.show()

Best trial: {'input_size': 29, 'n_stacks': 5, 'n_blocks': 4, 'max_steps': 17, 'local_scalar_type': None, 'interpretability': ['seasonality', 'trend']}


#### Try Out

In [44]:
for pollutant in data[['pm10','pm25','co','no2','o3']]:
    obj = TimeObject(df=data, column=pollutant, agg_freq='W')
    obj.NIXTLA_train_test(split=52)

    model = NBEATS(
        h=52,
        input_size=52,
        loss=DistributionLoss(distribution='StudentT', level=[95], return_params=True),
        stack_types=study_nbeats.best_trial.params.get('n_stacks')*['identity'],
        n_blocks =[study_nbeats.best_trial.params.get('n_blocks') for _ in range(study_nbeats.best_trial.params.get('n_stacks'))],
        max_steps=100,
        val_check_steps=3,
        learning_rate=1e-3
    )
    fcst = NeuralForecast(
        models=[model],
        freq='W',
        local_scaler_type=study_nbeats.best_trial.params.get('local_scaler_type')
    )
    fcst.fit(df=obj.Y_train, verbose=False)
    prediction = fcst.predict(df=obj.Y_train, verbose=False)

    fig = obj.plot_forecast(forecast_df=prediction, confidence=None, method='NBEATS', show=False)
    fig.update_layout({'width':1000, 'height':300})
    fig.show()

Seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type             | Params | Mode 
----------------------------------------------------------
0 | loss         | DistributionLoss | 3      | train
1 | padder_train | ConstantPad1d    | 0      | train
2 | scaler       | TemporalNorm     | 0      | train
3 | blocks       | ModuleList       | 18.4 M | train
----------------------------------------------------------
18.4 M    Trainable params
3         Non-trainable params
18.4 M    Total params
73.745    Total estimated model params size (MB)
184       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

mae: 9.4915
mape: 50.03015
mse: 125.50968
max: 21.59454
smape: 35.0196


# Others

### One

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import HoltWinters

obj = TimeObject(df=ponte[:], column='co', agg_freq='W')
obj.NIXTLA_train_test(split=52)

models = [HoltWinters(season_length=52, error_type="A", alias="Add"),
          HoltWinters(season_length=52, error_type="M", alias="Multi")]

models = StatsForecast(models=models, freq='W')
models.fit(obj.Y_train)
prediction = models.predict(h=52)

obj.plot_forecast(forecast_df=prediction, method='Add')

mae: 1.56395
mape: 42.05677
mse: 3.81552
max: 4.21615
smape: 44.16045


### Many

In [43]:
from statsforecast import StatsForecast
from statsforecast.models import HoltWinters

for pollutant in data[['pm10','pm25','co','no2','o3']]:

    obj = TimeObject(df=data[:], column=pollutant, agg_freq='W')
    obj.NIXTLA_train_test(split=52)

    models = [HoltWinters(season_length=52, error_type="A", alias="Add"),
            HoltWinters(season_length=52, error_type="M", alias="Multi")]

    models = StatsForecast(models=models, freq='W')
    models.fit(obj.Y_train)
    prediction = models.predict(h=52)

    fig = obj.plot_forecast(forecast_df=prediction, confidence=None, method='Add', show=False)
    fig.update_layout({'width':1000, 'height':300})
    fig.show()
    fig = obj.plot_forecast(forecast_df=prediction, confidence=None, method='Multi', show=False)
    fig.update_layout({'width':1000, 'height':300})
    fig.show()

mae: 4.83043
mape: 19.7223
mse: 40.86052
max: 13.87293
smape: 18.08294


mae: 5.05286
mape: 21.05321
mse: 43.20567
max: 15.05179
smape: 18.9799


mae: 11.28872
mape: 18.3942
mse: 189.22336
max: 38.31447
smape: 19.14318


mae: 9.42826
mape: 16.18626
mse: 159.76828
max: 30.67827
smape: 15.26206


mae: 1.56395
mape: 42.05677
mse: 3.81552
max: 4.21615
smape: 44.16045


mae: 1.59399
mape: 40.7394
mse: 3.96101
max: 4.98591
smape: 41.73753


mae: 4.52018
mape: 20.27477
mse: 33.67513
max: 17.53883
smape: 20.93117


mae: 4.59769
mape: 20.47409
mse: 34.27567
max: 16.86638
smape: 21.05063


mae: 6.42957
mape: 31.65183
mse: 66.01853
max: 22.68924
smape: 24.97801


mae: 6.48427
mape: 32.1358
mse: 67.78069
max: 21.51663
smape: 25.08003
